In [47]:
import pandas as pd

df = pd.read_csv("data/gaode_POI/高德POI分类与编码（中英文）_V1.06_20230208.CSV",encoding='GBK')
type_list_ = df['NEW_TYPE'].to_list()
type_list_str = '|'.join([str(item) for item in type_list_])
type_list_str

'10000|10100|10101|10102|10103|10104|10105|10107|10108|10109|10110|10111|10112|10200|10300|10400|10401|10500|10600|10700|10800|10900|10901|11000|11100|11101|11102|11103|20000|20100|20101|20102|20103|20104|20105|20106|20200|20201|20202|20203|20300|20301|20400|20401|20402|20403|20404|20405|20406|20407|20408|20600|20601|20602|20700|20701|20702|20703|20800|20900|20904|20905|21000|21001|21002|21003|21004|21100|21200|21201|21202|21203|21300|21301|21400|21401|21500|21501|21600|21601|21602|21700|21701|21702|21800|21802|21803|21804|21900|22000|22100|22200|22300|22301|22400|22500|22501|22502|22600|22700|22800|22900|23000|23100|23200|23300|23301|23400|23500|25000|25100|25200|25300|25400|25500|25600|25700|25800|25900|26000|26100|26200|26300|29900|30000|30100|30200|30201|30202|30203|30204|30205|30206|30300|30301|30302|30303|30400|30401|30500|30501|30502|30503|30504|30505|30506|30507|30508|30700|30701|30702|30800|30801|30802|30803|30900|31000|31004|31005|31100|31101|31102|31103|31104|31200|31300|313

In [48]:
import requests
import json
import xlwt
import math

# 改：TODO
amap_web_key = 'd2dfd4125abec352a2d85515e1789b0c'  # 高德地图官网申请的Web API KEY
filename = 'data/gaode_POI/POI_from_GD.xls'   # 爬取到的数据写入的EXCEL路径

# 改：多边形边界集合：
polygon_list = ['113.69911,22.757295|114.741955,22.488926']

# 改：POI分类集合, 多个类型用竖线 | 分割；
type_list = '140000'
# type_list =type_list_str

# '|060402|060403|060404|060405|060406|060407|060408|060409|060411|060413|060414|060415|060100|060101|060102|060103'


In [49]:

#=========================下面不用改=====================================

poi_search_url = "http://restapi.amap.com/v3/place/polygon"  # URL
offset = 25  # 分页请求数据时的单页大小
def gcj02_to_wgs84(lon, lat):
    a = 6378245.0
    ee = 0.00669342162296594323
    pi = 3.14159265358979324

    def transform_lon(x, y):
        ret = 300.0 + x + 2.0 * y + 0.1 * x * x + 0.1 * x * y + 0.1 * \
              y * y + (0.1 * x * x * x) + \
              (0.2 * x * x * y) + (0.1 * x * y * y) + \
              (0.1 * y * y * y) + (0.1 * x * x * x * x) + \
              (0.2 * x * x * x * y) + (0.2 * x * x * y * y) + \
              (0.1 * x * y * y * y)
        return ret

    def transform_lat(x, y):
        ret = -100.0 + 2.0 * x + 3.0 * y + 0.2 * y * \
              y + 0.1 * x * y + 0.2 * \
              x * x + (0.1 * x * x * x) + \
              (0.1 * x * y * y) + (0.1 * x * x * y)
        return ret

    def transform(x, y):
        d_lat = transform_lat(x - 105.0, y - 35.0)
        d_lon = transform_lon(x - 105.0, y - 35.0)
        rad_lat = y / 180.0 * pi
        magic = math.sin(rad_lat)
        magic = 1 - ee * magic * magic
        sqrt_magic = math.sqrt(magic)
        d_lat = (d_lat * 180.0) / ((a * (1 - ee)) / (magic * sqrt_magic) * pi)
        d_lon = (d_lon * 180.0) / (a / sqrt_magic * math.cos(rad_lat) * pi)
        mg_lat = y + d_lat
        mg_lon = x + d_lon
        return [x * 2 - mg_lon, y * 2 - mg_lat]

    return transform(lon, lat)


# 根据矩形坐标获取poi数据
def getpois(polygon, type_list):
    i = 1
    current_polygon_poi_list = []
    while True:  # 使用while循环不断分页获取数据
        result = getpoi_page(polygon, i, type_list)
        print(result)
        if not result.strip():  # 检查result是否为空或只包含空白字符
            print("结果为空，跳过当前迭代")
            i += 1
            continue  # 如果在循环中，使用continue跳过当前迭代
        result = json.loads(result)  # 将字符串转换为json

        if result['status'] != '1':  # 接口返回的状态不是1代表异常
            print('======爬取错误，返回数据：' + result)
            break
        pois = result['pois']
        if len(pois) < offset:  # 返回的数据不足分页页大小，代表数据爬取完
            current_polygon_poi_list.extend(pois)
            break
        current_polygon_poi_list.extend(pois)
        i += 1
    print('===========当前polygon：', polygon, ',爬取到的数据数量：', str(len(current_polygon_poi_list)))

    return current_polygon_poi_list


# 单页获取pois
def getpoi_page(polygon, page, type_list):
    # print(polygon)
    req_url = poi_search_url + "?key=" + amap_web_key + '&extensions=all&polygon=' + polygon + '&offset=' + str(
        offset) + '&types=' + type_list + '&page=' + str(page) + '&output=json'
    data = ''
    with requests.get(req_url) as response:
        data = response.text
        # print(data)
    return data


# 数据写入excel
def write_to_excel(poilist, filename):
    # 一个Workbook对象，这就相当于创建了一个Excel文件
    book = xlwt.Workbook(encoding='utf-8', style_compression=0)
    sheet = book.add_sheet('0', cell_overwrite_ok=True)
    # 第一行(列标题)

    sheet.write(0, 0, 'id')
    sheet.write(0, 1, 'name')
    sheet.write(0, 2, 'lon')
    sheet.write(0, 3, 'lat')
    sheet.write(0, 4, 'address')
    sheet.write(0, 5, 'pname')
    sheet.write(0, 6, 'cityname')
    sheet.write(0, 7, 'adname')
    sheet.write(0, 8, 'type')
    for i in range(len(poilist)):
        sheet.write(i + 1, 0, poilist[i]['id'])
        sheet.write(i + 1, 1, poilist[i]['name'])
        lon = float(str(poilist[i]['location']).split(",")[0])
        lat = float(str(poilist[i]['location']).split(",")[1])

        # 将高德坐标转换为WGS 84坐标
        lon, lat = gcj02_to_wgs84(lon, lat)

        sheet.write(i + 1, 2, lon)
        sheet.write(i + 1, 3, lat)
        sheet.write(i + 1, 4, poilist[i].get('address'))
        sheet.write(i + 1, 5, poilist[i].get('pname'))
        sheet.write(i + 1, 6, poilist[i].get('cityname'))
        sheet.write(i + 1, 7, poilist[i].get('adname'))
        sheet.write(i + 1, 8, poilist[i]['type'])

    book.save(filename)


print('开始爬取...')
all_poi_list = []  # 爬取到的所有数据

for polygon in polygon_list:
    polygon_poi_list = getpois(polygon, type_list)
    all_poi_list.extend(polygon_poi_list)

print('爬取完成，总的数量：', len(all_poi_list))

write_to_excel(all_poi_list, filename)

print('写入成功')


开始爬取...
{"status":"0","info":"USER_DAILY_QUERY_OVER_LIMIT","infocode":"10044"}


TypeError: can only concatenate str (not "dict") to str